In [1]:
##!pip install pydub


### 単音生成

In [18]:
import numpy as np
import scipy.io.wavfile as wavfile

# 12平均律に基づく、A4=440Hzを基準にした周波数計算
def get_frequency(note, octave):
    """
    与えられた音符とオクターブに基づいて周波数を計算する
    A4 = 440Hzを基準にして計算
    """
    # 音符名に対応する半音のオフセット
    note_offsets = {
        "C": -9, "C#": -8, "D": -7, "D#": -6, "E": -5, "F": -4,
        "F#": -3, "G": -2, "G#": -1, "A": 0, "A#": 1, "B": 2
    }
    
    # A4の周波数（基準となる）
    A4_freq = 440.0
    
    # オクターブのズレ（A4からの相対的なズレを計算）
    octave_shift = octave - 4  # A4が4オクターブにあるので、オクターブのズレを計算
    
    # 12平均律に基づく半音の比率（2^(1/12)）
    semitone_ratio = 2 ** (1 / 12)
    
    # Cからの半音のズレを計算
    semitone_offset = note_offsets[note]
    
    # 基準の周波数A4を基にして最終的な周波数を計算
    frequency = A4_freq * (semitone_ratio ** semitone_offset) * (2 ** octave_shift)
    
    return frequency

# C2からC5の音符とオクターブ
notes_and_octaves = [
    ("C", 2), ("C#", 2), ("D", 2), ("D#", 2), ("E", 2), ("F", 2), ("F#", 2), ("G", 2), ("G#", 2), ("A", 2), ("A#", 2), ("B", 2),
    ("C", 3), ("C#", 3), ("D", 3), ("D#", 3), ("E", 3), ("F", 3), ("F#", 3), ("G", 3), ("G#", 3), ("A", 3), ("A#", 3), ("B", 3),
    ("C", 4), ("C#", 4), ("D", 4), ("D#", 4), ("E", 4), ("F", 4), ("F#", 4), ("G", 4), ("G#", 4), ("A", 4), ("A#", 4), ("B", 4),
    ("C", 5)
]

"""
# 各音符の周波数を計算
note_frequencies = {}
for note, octave in notes_and_octaves:
    note_frequencies[f"{note}{octave}"] = get_frequency(note, octave)
"""

def generate_tone(frequency, duration, sample_rate=44100, amplitude=0.5):
    """
    与えられた周波数と長さに基づいて正弦波の音を生成する
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

def generate_chord_wav(chord, duration=1.0, sample_rate=44100, filename="output.wav"):
    """
    コード（和音）をWAVファイルとして生成する
    """
    full_wave = np.array([])

    for note in chord:
        frequency = note_frequencies[note]
        tone = generate_tone(frequency, duration, sample_rate)
        full_wave = np.concatenate((full_wave, tone))

    # 16bit PCMとしてWAVファイルを保存
    full_wave = np.int16(full_wave * 32767)  # 音量を調整して16bitに変換
    wavfile.write(filename, sample_rate, full_wave)
    print(f"WAVファイルが{filename}として保存されました")
    
"""
# 使用例: C2からC5までの単音を生成
for note in note_frequencies:
    generate_chord_wav([note], duration=0.5, filename=f"{note}_tone.wav")
"""

'\n# 使用例: C2からC5までの単音を生成\nfor note in note_frequencies:\n    generate_chord_wav([note], duration=0.5, filename=f"{note}_tone.wav")\n'

### コード音声生成

In [1]:
soundfiles = {
    "2" : {
        "C" : 'C2_tone.wav',
        "C#": 'C#2_tone.wav',
        "D" : 'D2_tone.wav',
        "D#": 'D#2_tone.wav',
        "E" : 'E2_tone.wav',
        "F" : 'F2_tone.wav',
        "F#": 'F#2_tone.wav',
        "G" : 'G2_tone.wav',
        "G#": 'G#2_tone.wav',
        "A" : 'A2_tone.wav',
        "A#": 'A#2_tone.wav',
        "B" : 'B2_tone.wav',
    },
    "3" : {
        "C" : 'C3_tone.wav',
        "C#": 'C#3_tone.wav',
        "D" : 'D3_tone.wav',
        "D#": 'D#3_tone.wav',
        "E" : 'E3_tone.wav',
        "F" : 'F3_tone.wav',
        "F#": 'F#3_tone.wav',
        "G" : 'G3_tone.wav',
        "G#": 'G#3_tone.wav',
        "A" : 'A3_tone.wav',
        "A#": 'A#3_tone.wav',
        "B" : 'B3_tone.wav',
    },
    "4" : {
        "C" : 'C4_tone.wav',
        "C#": 'C#4_tone.wav',
        "D" : 'D4_tone.wav',
        "D#": 'D#4_tone.wav',
        "E" : 'E4_tone.wav',
        "F" : 'F4_tone.wav',
        "F#": 'F#4_tone.wav',
        "G" : 'G4_tone.wav',
        "G#": 'G#4_tone.wav',
        "A" : 'A4_tone.wav',
        "A#": 'A#4_tone.wav',
        "B" : 'B4_tone.wav',
    },
    "5" : {
        "C" : 'C5_tone.wav',
    },
}

In [2]:
code_spans = {
    "maj":    [0, 4, 3],   # 例: C → E (4半音), E → G (3半音)
    "maj6":   [0, 4, 3, 2],  # 例: C E G A
    "dom7":   [0, 4, 3, 3],  # ドミナントセブンス（例: G7 = G B D F）
    "majadd9":[0, 4, 3, 7], # majに長9度を足したもの。
    "maj6-9": [0, 4, 3, 2, 5], # 長9度は長2度（全音一つ)の音階をオクターブ上か下に動かしたもの[-10, 0, 4, 3, 2]も同じ名前になっている
    "maj7":   [0, 4, 3, 4],  # 例: C E G B
    "maj7f5": [0, 4, 2, 4],  # 例: C E F# B 7th フラット5
    "maj9":   [0, 4, 3, 4, 3], # maj7に長9度を足したもの。
    "aug":  [0, 4, 4],   # オーグメント：拡張された、夢幻的な響き
    "min" : [0, 3, 4],   # 例: A → C (3), C → E (4)
    "dim" : [0, 3, 3],   # ディミニッシュ：不安定、緊張感
    "min7": [0, 3, 4, 3],  # 例: A C E G
    "dim7": [0, 3, 3, 3],  # ディミニッシュセブンス（完全に均等な構成）
    "m7b5": [0, 3, 3, 4],  # ハーフディミニッシュ（マイナー7♭5）
    "sus2": [0, 2, 5],     # サス2：ルート → 2度 → 完全5度
    "sus4": [0, 5, 2],     # サス4：ルート → 4度 → 完全5度
}

In [3]:
notes_and_octaves = [
    ("C", 2), ("C#", 2), ("D", 2), ("D#", 2), ("E", 2), ("F", 2), ("F#", 2), ("G", 2), ("G#", 2), ("A", 2), ("A#", 2), ("B", 2),
    ("C", 3), ("C#", 3), ("D", 3), ("D#", 3), ("E", 3), ("F", 3), ("F#", 3), ("G", 3), ("G#", 3), ("A", 3), ("A#", 3), ("B", 3),
    ("C", 4), ("C#", 4), ("D", 4), ("D#", 4), ("E", 4), ("F", 4), ("F#", 4), ("G", 4), ("G#", 4), ("A", 4), ("A#", 4), ("B", 4),
    ("C", 5)
]

### 主要なコードの構成音間の半音数

In [4]:
## コード音声生成。
# 関数：コード構成音を得るオクターブ含む
def get_chord_notes(root, code_type):
    root_index = NOTES.index(root)
    intervals = code_spans[code_type]
    notes = []
    idx = root_index
    for step in intervals:
        notes.append([NOTES[idx % 12],idx//12])
        idx += step
    return notes



In [5]:
import os
from pydub import AudioSegment
import os

# 出力用フォルダ
output_dir = "output_chords"
os.makedirs(output_dir, exist_ok=True)


In [12]:
# 音名リスト
NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
# コード生成（ルート音のオクターブ指定）
octave = 4
for root in NOTES:
    for code_type in code_spans:
        chord_notes = get_chord_notes(root, code_type)

        try:
            # 音声読み込み
            segments = [AudioSegment.from_wav("sounds/" + soundfiles[str(octave + bias)][note]) for [note, bias] in chord_notes]

            # 長さを揃えてミックス
            max_len = max(len(seg) for seg in segments)
            segments = [seg + AudioSegment.silent(duration=max_len - len(seg)) for seg in segments]

            # 重ねる
            chord_audio = segments[0]
            for seg in segments[1:]:
                chord_audio = chord_audio.overlay(seg)

            # 書き出し
            filename = f"octave{octave}_code_{root.replace('#', 's')}{code_type}.wav"  # ファイル名に "#" は使わない方が良い
            chord_audio.export(os.path.join(output_dir, filename), format="wav")
            print(f"✔️ Exported: {filename}")
        except KeyError as e:
            print(f"⚠️ Skipping {root} {code_type}: Missing sound file for {e}")

✔️ Exported: attenuation_octave4_code_Cmaj.wav
✔️ Exported: attenuation_octave4_code_Cmaj6.wav
✔️ Exported: attenuation_octave4_code_Cdom7.wav
✔️ Exported: attenuation_octave4_code_Cmajadd9.wav
✔️ Exported: attenuation_octave4_code_Cmaj6-9.wav
✔️ Exported: attenuation_octave4_code_Cmaj7.wav
✔️ Exported: attenuation_octave4_code_Cmaj7f5.wav
✔️ Exported: attenuation_octave4_code_Cmaj9.wav
✔️ Exported: attenuation_octave4_code_Caug.wav
✔️ Exported: attenuation_octave4_code_Cmin.wav
✔️ Exported: attenuation_octave4_code_Cdim.wav
✔️ Exported: attenuation_octave4_code_Cmin7.wav
✔️ Exported: attenuation_octave4_code_Cdim7.wav
✔️ Exported: attenuation_octave4_code_Cm7b5.wav
✔️ Exported: attenuation_octave4_code_Csus2.wav
✔️ Exported: attenuation_octave4_code_Csus4.wav
✔️ Exported: attenuation_octave4_code_Csmaj.wav
✔️ Exported: attenuation_octave4_code_Csmaj6.wav
✔️ Exported: attenuation_octave4_code_Csdom7.wav
✔️ Exported: attenuation_octave4_code_Csmajadd9.wav
✔️ Exported: attenuation_octave